In [1]:
!pip uninstall qiskit qiskit-aer qiskit-algorithms qiskit-optimization -y

# Step 2: Install compatible versions (tested combination)
!pip install qiskit==1.0.2
!pip install qiskit-aer==0.14.2
!pip install qiskit-algorithms==0.3.0
!pip install qiskit-optimization==0.6.1

Found existing installation: qiskit 1.0.2
Uninstalling qiskit-1.0.2:
  Successfully uninstalled qiskit-1.0.2
  Using cached qiskit-1.0.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
Using cached qiskit-1.0.2-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (5.6 MB)
  Using cached qiskit_aer-0.14.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.1 kB)
Using cached qiskit_aer-0.14.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.6/308.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 646.5/646.5 kB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.6/167.6 kB 10.6 MB/s eta 0:00:00
  Created wheel for docplex: filename=docplex-2.30.251-py3-none-any

In [ ]:
import numpy as np
import random
from itertools import permutations, combinations

# Set seed for reproducible results
random.seed(42)
np.random.seed(42)

# Enhanced TSP solver with FIXED quantum implementation
QUANTUM_AVAILABLE = False
try:
    from qiskit_algorithms import QAOA
    from qiskit_algorithms.optimizers import SPSA, COBYLA
    from qiskit_optimization import QuadraticProgram
    from qiskit_optimization.algorithms import MinimumEigenOptimizer

    try:
        from qiskit_aer.primitives import Sampler
        QUANTUM_AVAILABLE = True
        print("✓ Quantum libraries available - QAOA enabled")
    except ImportError:
        try:
            from qiskit.primitives import Sampler
            QUANTUM_AVAILABLE = True
            print("✓ Quantum libraries available - QAOA enabled")
        except ImportError:
            print("✗ Quantum sampler not available")
except ImportError:
    print("✗ Quantum libraries not available - classical only")

def create_challenging_6city_tsp():
    """Create a 6-city TSP instance designed to show quantum vs classical differences"""
    n_nodes = 6

    # Cost matrix: Asymmetric with multiple local optima
    cost_matrix = [
        [0,  12, 18, 25, 22, 30],  # Hub -> others
        [15, 0,  28, 20, 35, 16],  # Airport -> others
        [20, 32, 0,  14, 26, 24],  # Port -> others
        [28, 22, 16, 0,  19, 21],  # Mall -> others
        [25, 38, 29, 17, 0,  33],  # University -> others
        [35, 18, 27, 23, 36, 0]   # Hospital -> others
    ]

    # Passenger matrix: Creates conflicts with cost optimization
    passenger_matrix = [
        [0,  15, 12, 8,  18, 6 ],  # Hub: High to University, Airport
        [12, 0,  5,  22, 9,  25],  # Airport: High to Hospital, Mall
        [10, 8,  0,  20, 14, 11],  # Port: High to Mall, University
        [7,  19, 18, 0,  16, 13],  # Mall: High to Airport, Port
        [20, 11, 15, 17, 0,  7 ],  # University: High to Hub, Mall
        [5,  28, 13, 14, 8,  0 ]   # Hospital: High to Airport
    ]

    return n_nodes, cost_matrix, passenger_matrix

def calculate_route_score(route, cost_matrix, passenger_matrix, cost_weight=1.0, passenger_weight=1.0):
    """Calculate score for a route: minimize cost, maximize passengers"""
    total_cost = 0
    total_passengers = 0

    for i in range(len(route)):
        start = route[i]
        end = route[(i + 1) % len(route)]
        total_cost += cost_matrix[start][end]
        total_passengers += passenger_matrix[start][end]

    score = cost_weight * total_cost - passenger_weight * total_passengers
    return score, total_cost, total_passengers

def brute_force_optimal(n_nodes, cost_matrix, passenger_matrix, cost_weight=1.0, passenger_weight=1.0):
    """Find the true optimal solution by brute force"""
    if n_nodes > 7:
        print("Brute force not feasible for >7 cities")
        return None

    print(f"Finding optimal solution by brute force...")

    best_route = None
    best_score = float('inf')
    best_cost = 0
    best_passengers = 0

    # Fix first city as 0, permute the rest
    for perm in permutations(range(1, n_nodes)):
        route = [0] + list(perm)
        score, cost, passengers = calculate_route_score(
            route, cost_matrix, passenger_matrix, cost_weight, passenger_weight
        )

        if score < best_score:
            best_score = score
            best_route = route
            best_cost = cost
            best_passengers = passengers

    return {
        'route': best_route,
        'score': best_score,
        'cost': best_cost,
        'passengers': best_passengers
    }

def generate_random_routes(n_nodes, num_routes=100):
    """Generate random valid TSP routes"""
    routes = []
    base_route = list(range(n_nodes))

    for _ in range(num_routes):
        route = base_route.copy()
        remaining = route[1:]  # Keep first city fixed
        random.shuffle(remaining)
        routes.append([route[0]] + remaining)

    return routes

def local_search_2opt(route, cost_matrix, passenger_matrix, cost_weight, passenger_weight):
    """2-opt improvement with first improvement strategy"""
    best_route = route.copy()
    improved = True

    while improved:
        improved = False
        best_score, _, _ = calculate_route_score(best_route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

        for i in range(1, len(route) - 1):
            for j in range(i + 1, len(route)):
                new_route = best_route.copy()
                new_route[i:j+1] = reversed(new_route[i:j+1])

                new_score, _, _ = calculate_route_score(new_route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

                if new_score < best_score:
                    best_route = new_route
                    best_score = new_score
                    improved = True
                    break
            if improved:
                break

    return best_route

def enhanced_classical_solver(problem, num_random_routes=2000, use_multiple_starts=True):
    """Enhanced classical solver with multiple strategies"""
    n_nodes = problem['n_nodes']
    cost_matrix = problem['cost_matrix']
    passenger_matrix = problem['passenger_matrix']
    cost_weight = problem['cost_weight']
    passenger_weight = problem['passenger_weight']

    print(f"Enhanced classical solver: {num_random_routes} random + local search")

    best_route = None
    best_score = float('inf')

    # Strategy 1: Random + 2-opt
    routes = generate_random_routes(n_nodes, num_random_routes)
    for route in routes:
        improved_route = local_search_2opt(route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)
        score, cost, passengers = calculate_route_score(improved_route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

        if score < best_score:
            best_score = score
            best_route = improved_route

    # Strategy 2: Greedy construction
    if use_multiple_starts:
        for start_city in range(n_nodes):
            greedy_route = greedy_construction(start_city, cost_matrix, passenger_matrix, cost_weight, passenger_weight)
            improved_route = local_search_2opt(greedy_route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)
            score, cost, passengers = calculate_route_score(improved_route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

            if score < best_score:
                best_score = score
                best_route = improved_route

    final_score, final_cost, final_passengers = calculate_route_score(best_route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

    return {
        'route': best_route,
        'score': final_score,
        'cost': final_cost,
        'passengers': final_passengers
    }

def greedy_construction(start_city, cost_matrix, passenger_matrix, cost_weight, passenger_weight):
    """Greedy TSP construction starting from a specific city"""
    n_nodes = len(cost_matrix)
    route = [start_city]
    unvisited = set(range(n_nodes)) - {start_city}

    current = start_city
    while unvisited:
        best_next = None
        best_value = float('inf')

        for next_city in unvisited:
            value = (cost_weight * cost_matrix[current][next_city] -
                    passenger_weight * passenger_matrix[current][next_city])
            if value < best_value:
                best_value = value
                best_next = next_city

        route.append(best_next)
        unvisited.remove(best_next)
        current = best_next

    return route

def create_quantum_tsp_with_subtour_elimination(problem):
    """
    Create quantum TSP formulation with PROPER subtour elimination using penalty terms
    """
    n_nodes = problem['n_nodes']
    cost_matrix = problem['cost_matrix']
    passenger_matrix = problem['passenger_matrix']
    cost_weight = problem['cost_weight']
    passenger_weight = problem['passenger_weight']

    qp = QuadraticProgram()

    # Create binary variables x[i][j] = 1 if we go from city i to city j
    var_names = {}
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j:
                var_name = f"x_{i}_{j}"
                qp.binary_var(name=var_name)
                var_names[(i, j)] = var_name

    # Objective function: minimize cost - maximize passengers
    linear_terms = {}
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j:
                var_name = var_names[(i, j)]
                coefficient = (cost_weight * cost_matrix[i][j] -
                             passenger_weight * passenger_matrix[i][j])
                linear_terms[var_name] = coefficient

    qp.minimize(linear=linear_terms)

    # Constraint 1: Each city has exactly one outgoing edge
    for i in range(n_nodes):
        constraint_vars = {}
        for j in range(n_nodes):
            if i != j:
                constraint_vars[var_names[(i, j)]] = 1
        qp.linear_constraint(linear=constraint_vars, sense="==", rhs=1, name=f"out_{i}")

    # Constraint 2: Each city has exactly one incoming edge
    for j in range(n_nodes):
        constraint_vars = {}
        for i in range(n_nodes):
            if i != j:
                constraint_vars[var_names[(i, j)]] = 1
        qp.linear_constraint(linear=constraint_vars, sense="==", rhs=1, name=f"in_{j}")

    # NEW: Add quadratic penalty terms to discourage subtours
    # This is the KEY fix that was missing!
    quadratic_terms = {}
    penalty_strength = max(max(row) for row in cost_matrix) * 2  # Strong penalty

    # Penalty for 2-city subtours
    for i in range(n_nodes):
        for j in range(n_nodes):
            if i != j and (i, j) in var_names and (j, i) in var_names:
                # Penalize x[i][j] * x[j][i] (2-city cycles)
                quadratic_terms[(var_names[(i, j)], var_names[(j, i)])] = penalty_strength

    # Add additional penalties for small subtours
    for subset_size in [3, 4]:  # Penalize 3 and 4-city subtours
        if subset_size < n_nodes:
            for subset in combinations(range(n_nodes), subset_size):
                # Add penalty terms for edges within this subset forming a cycle
                subset_edges = []
                for i in subset:
                    for j in subset:
                        if i != j and (i, j) in var_names:
                            subset_edges.append(var_names[(i, j)])

                # Add cross terms to discourage internal cycles
                for i, edge1 in enumerate(subset_edges):
                    for j, edge2 in enumerate(subset_edges[i+1:], i+1):
                        if (edge1, edge2) not in quadratic_terms:
                            quadratic_terms[(edge1, edge2)] = penalty_strength * 0.1

    if quadratic_terms:
        qp.minimize(linear=linear_terms, quadratic=quadratic_terms)
        print(f"Added {len(quadratic_terms)} quadratic penalty terms for subtour elimination")

    return qp, var_names

def solve_quantum_tsp_fixed(problem, reps=3, maxiter=200):
    """
    FIXED quantum TSP solver with proper subtour elimination
    """
    if not QUANTUM_AVAILABLE:
        print("Quantum solver not available, using classical")
        return enhanced_classical_solver(problem)

    try:
        n_nodes = problem['n_nodes']
        cost_matrix = problem['cost_matrix']
        passenger_matrix = problem['passenger_matrix']
        cost_weight = problem['cost_weight']
        passenger_weight = problem['passenger_weight']

        print(f"FIXED Quantum QAOA with subtour elimination: {reps} layers, {maxiter} iterations")

        # Create proper quantum formulation
        qp, var_names = create_quantum_tsp_with_subtour_elimination(problem)

        # Get sampler
        try:
            from qiskit_aer.primitives import Sampler
            sampler = Sampler()
        except ImportError:
            from qiskit.primitives import Sampler
            sampler = Sampler()

        # Try different QAOA configurations
        configs = [
            ("SPSA-Aggressive", SPSA(maxiter=maxiter, learning_rate=0.05, perturbation=0.1)),
            ("SPSA-Conservative", SPSA(maxiter=maxiter//2, learning_rate=0.1, perturbation=0.05)),
            ("COBYLA", COBYLA(maxiter=maxiter//3))
        ]

        best_result = None
        best_score = float('inf')

        for config_name, optimizer in configs:
            try:
                print(f"Trying {config_name}...")
                qaoa = QAOA(sampler=sampler, optimizer=optimizer, reps=reps)
                algo = MinimumEigenOptimizer(qaoa)

                result = algo.solve(qp)

                if hasattr(result, 'x') and result.x is not None:
                    print(f"QAOA solution vector length: {len(result.x)}")
                    print(f"Non-zero elements: {sum(1 for x in result.x if abs(x) > 0.1)}")

                    # Extract route with IMPROVED method
                    route = extract_route_robust(result.x, var_names, n_nodes, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

                    if route and len(set(route)) == n_nodes:  # Valid route check
                        # Apply local search improvement
                        route = local_search_2opt(route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

                        score, cost, passengers = calculate_route_score(route, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

                        print(f"{config_name}: Route {route}, Cost {cost}, Passengers {passengers}, Score {score:.3f}")

                        if score < best_score:
                            best_score = score
                            best_result = {
                                'route': route,
                                'score': score,
                                'cost': cost,
                                'passengers': passengers,
                                'optimizer': config_name,
                                'quantum_objective': result.fval if hasattr(result, 'fval') else None
                            }
                    else:
                        print(f"{config_name}: Invalid route extracted: {route}")

            except Exception as e:
                print(f"{config_name} failed: {e}")
                continue

        if best_result:
            print(f"Best quantum result: {best_result['optimizer']} with score {best_result['score']:.3f}")
            return best_result
        else:
            print("All quantum configurations failed, using classical fallback")
            return enhanced_classical_solver(problem)

    except Exception as e:
        print(f"Quantum solver completely failed: {e}")
        return enhanced_classical_solver(problem)

def extract_route_robust(solution, var_names, n_nodes, cost_matrix, passenger_matrix, cost_weight, passenger_weight):
    """
    ROBUST route extraction that tries multiple strategies
    """
    print("Extracting route using robust method...")

    # Strategy 1: Extract high-confidence edges
    var_list = list(var_names.keys())
    high_confidence_edges = []
    medium_confidence_edges = []

    for idx, value in enumerate(solution):
        if idx < len(var_list):
            edge = var_list[idx]
            if value > 0.7:  # High confidence
                high_confidence_edges.append(edge)
            elif value > 0.3:  # Medium confidence
                medium_confidence_edges.append(edge)

    print(f"High confidence edges (>0.7): {high_confidence_edges}")
    print(f"Medium confidence edges (>0.3): {medium_confidence_edges}")

    # Strategy 1: Try to build from high confidence edges
    route = build_route_from_edges(high_confidence_edges, n_nodes)
    if is_valid_complete_route(route, n_nodes):
        print("Successfully built route from high confidence edges")
        return route

    # Strategy 2: Try with medium confidence edges
    all_edges = high_confidence_edges + medium_confidence_edges
    route = build_route_from_edges(all_edges, n_nodes)
    if is_valid_complete_route(route, n_nodes):
        print("Successfully built route from medium confidence edges")
        return route

    # Strategy 3: Probabilistic construction based on solution values
    print("Using probabilistic route construction...")
    route = probabilistic_route_construction(solution, var_names, n_nodes)
    if is_valid_complete_route(route, n_nodes):
        return route

    # Strategy 4: Generate multiple candidates and pick best
    print("Generating multiple candidate routes...")
    candidates = []

    # Try different thresholds
    for threshold in [0.2, 0.1, 0.05]:
        edges = []
        for idx, value in enumerate(solution):
            if idx < len(var_list) and value > threshold:
                edges.append(var_list[idx])

        candidate = build_route_from_edges(edges, n_nodes)
        if is_valid_complete_route(candidate, n_nodes):
            score, _, _ = calculate_route_score(candidate, cost_matrix, passenger_matrix, cost_weight, passenger_weight)
            candidates.append((candidate, score))

    if candidates:
        best_candidate = min(candidates, key=lambda x: x[1])
        print(f"Best candidate route: {best_candidate[0]} with score {best_candidate[1]:.3f}")
        return best_candidate[0]

    # Strategy 5: Fallback to a good heuristic route (NOT sequential!)
    print("All extraction methods failed, using greedy fallback")
    return greedy_construction(0, cost_matrix, passenger_matrix, cost_weight, passenger_weight)

def probabilistic_route_construction(solution, var_names, n_nodes):
    """Build route by selecting edges probabilistically based on solution values"""
    var_list = list(var_names.keys())

    # Create probability distribution for edges
    edge_probs = {}
    for idx, value in enumerate(solution):
        if idx < len(var_list):
            edge = var_list[idx]
            edge_probs[edge] = max(0, value)  # Ensure non-negative

    # Normalize probabilities for each city's outgoing edges
    route = [0]  # Start at city 0
    current = 0
    visited = {0}

    while len(route) < n_nodes and len(visited) < n_nodes:
        # Get outgoing edges from current city
        outgoing = [(edge, prob) for edge, prob in edge_probs.items()
                   if edge[0] == current and edge[1] not in visited]

        if not outgoing:
            # No valid outgoing edges, pick unvisited city with highest overall probability
            unvisited = [city for city in range(n_nodes) if city not in visited]
            if unvisited:
                # Find city with highest total incoming probability
                best_city = max(unvisited, key=lambda city: sum(
                    edge_probs.get((i, city), 0) for i in range(n_nodes) if i != city
                ))
                route.append(best_city)
                visited.add(best_city)
                current = best_city
            else:
                break
        else:
            # Choose edge probabilistically (weighted by solution values)
            total_prob = sum(prob for _, prob in outgoing)
            if total_prob > 0:
                rand_val = random.random() * total_prob
                cumulative = 0
                chosen_edge = outgoing[0][0]  # fallback

                for edge, prob in outgoing:
                    cumulative += prob
                    if rand_val <= cumulative:
                        chosen_edge = edge
                        break

                next_city = chosen_edge[1]
                route.append(next_city)
                visited.add(next_city)
                current = next_city
            else:
                # All probabilities are 0, pick randomly
                unvisited = [edge[1] for edge, _ in outgoing]
                if unvisited:
                    next_city = random.choice(unvisited)
                    route.append(next_city)
                    visited.add(next_city)
                    current = next_city

    # Fill in any missing cities
    missing = [city for city in range(n_nodes) if city not in route]
    route.extend(missing)

    return route[:n_nodes]

def build_route_from_edges(edges, n_nodes):
    """Build a route from a list of edges"""
    if not edges:
        return list(range(n_nodes))

    # Create adjacency mapping
    next_city = {}
    for start, end in edges:
        if start not in next_city:  # Take first edge if multiple from same city
            next_city[start] = end

    # Try to build route starting from city 0
    route = []
    current = 0
    visited = set()

    while len(route) < n_nodes and current not in visited:
        route.append(current)
        visited.add(current)
        if current in next_city:
            current = next_city[current]
        else:
            # Try to find any unvisited city that has an outgoing edge
            remaining_starts = [start for start, end in edges if start not in visited]
            if remaining_starts:
                current = remaining_starts[0]
            else:
                break

    # Fill in missing cities
    missing = [i for i in range(n_nodes) if i not in route]
    route.extend(missing)

    return route[:n_nodes]

def is_valid_complete_route(route, n_nodes):
    """Check if route visits all cities exactly once"""
    return (route is not None and
            len(route) == n_nodes and
            len(set(route)) == n_nodes and
            all(0 <= city < n_nodes for city in route))

def comprehensive_comparison_fixed():
    """Run comprehensive comparison with FIXED quantum implementation"""
    print("="*80)
    print("COMPREHENSIVE 6-CITY TSP COMPARISON (FIXED QUANTUM)")
    print("="*80)

    # Create the challenging 6-city instance
    n_nodes, cost_matrix, passenger_matrix = create_challenging_6city_tsp()

    print(f"\n6-City TSP Instance")
    print("\nCost Matrix:")
    cities = ["Hub", "Airport", "Port", "Mall", "Univ", "Hospital"]
    print("From\\To   " + "".join(f"{city:>8}" for city in cities))
    for i, row in enumerate(cost_matrix):
        print(f"{cities[i]:<8} " + "".join(f"{cost:>8}" for cost in row))

    print("\nPassenger Matrix:")
    print("From\\To   " + "".join(f"{city:>8}" for city in cities))
    for i, row in enumerate(passenger_matrix):
        print(f"{cities[i]:<8} " + "".join(f"{passengers:>8}" for passengers in row))

    # Test the most interesting strategy
    strategies = [
        (1.0, 1.0, "Balanced"),
        (1.0, 2.0, "Passenger-Focused"),
    ]

    for cost_weight, passenger_weight, strategy_name in strategies:
        print(f"\n{'='*60}")
        print(f"STRATEGY: {strategy_name}")
        print(f"Cost Weight: {cost_weight}, Passenger Weight: {passenger_weight}")
        print(f"{'='*60}")

        problem = {
            'n_nodes': n_nodes,
            'cost_matrix': cost_matrix,
            'passenger_matrix': passenger_matrix,
            'cost_weight': cost_weight,
            'passenger_weight': passenger_weight
        }

        # 1. Brute Force Optimal
        print("\n--- BRUTE FORCE OPTIMAL ---")
        optimal = brute_force_optimal(n_nodes, cost_matrix, passenger_matrix, cost_weight, passenger_weight)
        print(f"Optimal: Route {optimal['route']}, Cost {optimal['cost']}, Passengers {optimal['passengers']}, Score {optimal['score']:.3f}")

        # 2. Classical
        print("\n--- CLASSICAL SOLVER ---")
        classical = enhanced_classical_solver(problem, num_random_routes=2000)
        classical_gap = ((classical['score'] - optimal['score']) / optimal['score']) * 100
        print(f"Classical: Route {classical['route']}, Cost {classical['cost']}, Passengers {classical['passengers']}, Score {classical['score']:.3f}")
        print(f"Classical gap from optimal: {classical_gap:.2f}%")

        # 3. FIXED Quantum
        if QUANTUM_AVAILABLE:
            print("\n--- FIXED QUANTUM QAOA ---")
            quantum = solve_quantum_tsp_fixed(problem, reps=3, maxiter=250)
            if quantum:
                quantum_gap = ((quantum['score'] - optimal['score']) / optimal['score']) * 100
                print(f"Quantum: Route {quantum['route']}, Cost {quantum['cost']}, Passengers {quantum['passengers']}, Score {quantum['score']:.3f}")
                print(f"Quantum gap from optimal: {quantum_gap:.2f}%")

                # Direct comparison
                if quantum['score'] < classical['score']:
                    improvement = ((classical['score'] - quantum['score']) / classical['score']) * 100
                    print(f"🎉 QUANTUM BEATS CLASSICAL by {improvement:.2f}%!")
                elif quantum['score'] > classical['score']:
                    degradation = ((quantum['score'] - classical['score']) / classical['score']) * 100
                    print(f"❌ Quantum {degradation:.2f}% worse than classical")
                else:
                    print("⚖️ Quantum and classical tied")

                # Show if routes are different
                if quantum['route'] != classical['route']:
                    print(f"✨ DIFFERENT ROUTES FOUND!")
                    print(f"   Classical route: {classical['route']}")
                    print(f"   Quantum route:   {quantum['route']}")
                else:
                    print("Routes are identical")
            else:
                print("Quantum solver failed completely")
        else:
            print("Quantum solver not available")

if __name__ == "__main__":
    comprehensive_comparison_fixed()